# Appendix A - Intro to PyTorch

In [16]:
# import libraries
import torch
print(torch.__version__)
print(torch.cuda.is_available())

2.4.0+cpu
False


### A.2

In [17]:
import torch.nn.functional as F

y = torch.tensor([1.0])
x1 = torch.tensor([1.1])
w1 = torch.tensor([2.2])
b = torch.tensor([0.0])

z = x1 * w1 + b
a = torch.sigmoid(z)
loss = F.binary_cross_entropy(a,y)

### A.3

In [18]:
import torch.nn.functional as F
from torch.autograd import grad

y = torch.tensor([1.0])
x1 = torch.tensor([1.1])
w1 = torch.tensor([2.2], requires_grad = True)
b = torch.tensor([0.0], requires_grad = True)

z = x1 * w1 + b
a = torch.sigmoid(z)
loss = F.binary_cross_entropy(a,y)

grad_L_w1 = grad(loss, w1, retain_graph=True)
grad_L_b = grad(loss, b, retain_graph=True)

print(grad_L_w1)
print(grad_L_b)

(tensor([-0.0898]),)
(tensor([-0.0817]),)


In [19]:
loss.backward()
print(grad_L_w1)
print(grad_L_b)

(tensor([-0.0898]),)
(tensor([-0.0817]),)


### A.4

In [20]:
class NeuralNetwork(torch.nn.Module):
    def __init__(self, num_inputs, num_outputs):
        super().__init__()

        self.layers = torch.nn.Sequential (
    
            # 1st hiden layer
            torch.nn.Linear(num_inputs, 30),
            torch.nn.ReLU(),
    
            # 2nd hidden layer
            torch.nn.Linear(30, 20),
            torch.nn.ReLU(),
    
            # output layer
            torch.nn.Linear(20, num_outputs),
        )

    def forward(self, x):
        logits = self.layers(x)
        return logits

model = NeuralNetwork(50,3)
print(model)

NeuralNetwork(
  (layers): Sequential(
    (0): Linear(in_features=50, out_features=30, bias=True)
    (1): ReLU()
    (2): Linear(in_features=30, out_features=20, bias=True)
    (3): ReLU()
    (4): Linear(in_features=20, out_features=3, bias=True)
  )
)


In [21]:
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print("Total trainable params:", num_params)

Total trainable params: 2213


In [22]:
print(model.layers[0].weight)

Parameter containing:
tensor([[ 0.0502,  0.0307,  0.0333,  ...,  0.0951,  0.1134, -0.0297],
        [ 0.1077, -0.1108,  0.0122,  ...,  0.0108, -0.1049, -0.1063],
        [-0.0920, -0.0480,  0.0105,  ..., -0.0923,  0.1201,  0.0330],
        ...,
        [ 0.1359,  0.0175, -0.0673,  ...,  0.0674,  0.0676,  0.1058],
        [ 0.0790,  0.1343, -0.0293,  ...,  0.0344, -0.0971, -0.0509],
        [-0.1250,  0.0513,  0.0366,  ..., -0.1370,  0.1074, -0.0704]],
       requires_grad=True)


In [23]:
print(model.layers[0].weight.shape)

torch.Size([30, 50])


In [24]:
print(model.layers[0].bias)

Parameter containing:
tensor([-1.3122e-01, -1.0667e-02,  1.0314e-01, -1.8104e-02,  3.1523e-02,
        -1.3763e-01,  1.1643e-01,  9.1198e-02,  9.8382e-02,  7.3479e-02,
        -7.2337e-02, -1.1853e-01,  5.3997e-04,  7.5849e-02, -6.1513e-02,
         3.5053e-02, -1.1154e-02, -1.3147e-02,  3.6492e-02,  1.0322e-01,
         2.9582e-02,  1.0176e-02,  2.2896e-02,  2.6020e-02, -4.5835e-02,
        -1.6127e-02,  3.4467e-02, -1.1141e-01,  9.3445e-05, -1.3079e-01],
       requires_grad=True)


In [25]:
torch.manual_seed(123)
model = NeuralNetwork(50,3)
print(model.layers[0].weight)

Parameter containing:
tensor([[-0.0577,  0.0047, -0.0702,  ...,  0.0222,  0.1260,  0.0865],
        [ 0.0502,  0.0307,  0.0333,  ...,  0.0951,  0.1134, -0.0297],
        [ 0.1077, -0.1108,  0.0122,  ...,  0.0108, -0.1049, -0.1063],
        ...,
        [-0.0787,  0.1259,  0.0803,  ...,  0.1218,  0.1303, -0.1351],
        [ 0.1359,  0.0175, -0.0673,  ...,  0.0674,  0.0676,  0.1058],
        [ 0.0790,  0.1343, -0.0293,  ...,  0.0344, -0.0971, -0.0509]],
       requires_grad=True)


In [26]:
torch.manual_seed(123)
X = torch.rand([1,50])
out = model(X)
print(out)

tensor([[-0.1262,  0.1080, -0.1792]], grad_fn=<AddmmBackward0>)


In [27]:
with torch.no_grad():
    out=model(X)
print(out)

tensor([[-0.1262,  0.1080, -0.1792]])


In [28]:
with torch.no_grad():
    out=torch.softmax(model(X), dim=1)
print(out)

tensor([[0.3113, 0.3934, 0.2952]])


### A.5

In [29]:
X_train = torch.tensor([
    [-1.2, 1.1],
    [-0.9, 2.9],
    [-0.5, 2.6],
    [2.3, -1.1],
    [2.7, -1.5]
])

y_train = torch.tensor([0,0,0,1,1])

X_test = torch.tensor([
    [-0.8, 2.8],
    [2.6, -1.6]
])

y_test = torch.tensor([0,1])

### A.6

In [30]:
from torch.utils.data import Dataset

class ToyDataset(Dataset):
    def __init__(self, X, y):
        self.features = X
        self.labels=y

    def __getitem__(self, index):
        one_x = self.features[index]
        one_y = self.labels[index]
        return one_x, one_y

    def __len__(self):
        return self.labels.shape[0]

train_ds = ToyDataset(X_train, y_train)
test_ds = ToyDataset(X_test, y_test)

In [31]:
print(len(train_ds))

5


### A.7

In [32]:
from torch.utils.data import DataLoader

torch.manual_seed(123)

train_loader = DataLoader(
    dataset=train_ds,
    batch_size=2,
    shuffle=True,
    num_workers=0
)

test_loader = DataLoader(
    dataset=test_ds,
    batch_size=2,
    shuffle=False,
    num_workers=0
)

In [33]:
for idx, (x,y) in enumerate(train_loader):
    print(f"Batch {idx+1}:", x, y)

Batch 1: tensor([[ 2.3000, -1.1000],
        [-0.9000,  2.9000]]) tensor([1, 0])
Batch 2: tensor([[-1.2000,  1.1000],
        [-0.5000,  2.6000]]) tensor([0, 0])
Batch 3: tensor([[ 2.7000, -1.5000]]) tensor([1])


In [34]:
train_loader = DataLoader(
    dataset=train_ds,
    batch_size=2,
    shuffle=True,
    num_workers=0,
    drop_last=True
)

In [35]:
for idx, (x,y) in enumerate(train_loader):
    print(f"Batch {idx+1}:", x, y)

Batch 1: tensor([[-1.2000,  1.1000],
        [-0.5000,  2.6000]]) tensor([0, 0])
Batch 2: tensor([[ 2.3000, -1.1000],
        [-0.9000,  2.9000]]) tensor([1, 0])


### A.8

In [36]:
import torch.nn.functional as F

torch.manual_seed(123)

model = NeuralNetwork(num_inputs=2, num_outputs=2)
optimizer = torch.optim.SGD(
    model.parameters(), lr=0.5
)

num_epochs=3

for epoch in range(num_epochs):
    model.train()
    for batch_idx, (features, labels) in enumerate(train_loader):
        logits = model(features)
        loss = F.cross_entropy(logits, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # logging
        print(f"Epoch: {epoch+1:03d}/{num_epochs:03d}"
             f" Batch {batch_idx:03d}/{len(train_loader):03d}"
             f" Train loss: {loss:.2f}"
        )
        
    model.eval()


Epoch: 001/003 Batch 000/002 Train loss: 0.75
Epoch: 001/003 Batch 001/002 Train loss: 0.69
Epoch: 002/003 Batch 000/002 Train loss: 0.46
Epoch: 002/003 Batch 001/002 Train loss: 0.08
Epoch: 003/003 Batch 000/002 Train loss: 0.05
Epoch: 003/003 Batch 001/002 Train loss: 0.00


In [38]:
# do evaluations
model.eval()
with torch.no_grad():
    outputs = model(X_train)
print(outputs)

tensor([[ 1.4820, -2.1766],
        [ 2.5881, -3.8108],
        [ 2.1284, -3.2190],
        [-1.4925,  1.4908],
        [-1.7317,  1.7459]])


In [41]:
torch.set_printoptions(sci_mode=False)
probas = torch.softmax(outputs, dim=1)
print(probas)

tensor([[0.9749, 0.0251],
        [0.9983, 0.0017],
        [0.9953, 0.0047],
        [0.0482, 0.9518],
        [0.0300, 0.9700]])


In [42]:
predictions = torch.argmax(probas, dim=1)
print(predictions)

tensor([0, 0, 0, 1, 1])


In [43]:
predictions = torch.argmax(outputs, dim=1)
print(predictions)

tensor([0, 0, 0, 1, 1])


In [44]:
predictions == y_train

tensor([True, True, True, True, True])

In [45]:
torch.sum(predictions==y_train)

tensor(5)

In [50]:
def compute_accuracy(model, dataloader):
    model = model.eval()
    correct =0.0
    total_examples = 0 

    for idx, (features, labels) in enumerate(dataloader):
        with torch.no_grad():
            logits = model(features)

        predictions = torch.argmax(logits, dim=1)
        compare = labels==predictions
        correct += torch.sum(compare)
        total_examples += len(compare)

    return (correct / total_examples).item()

In [51]:
print(compute_accuracy(model, train_loader))

1.0


In [52]:
print(compute_accuracy(model, test_loader))

1.0


In [53]:
torch.save(model.state_dict(), "model.pth")

In [54]:
model = NeuralNetwork(2,2)
model.load_state_dict(torch.load("model.pth"))

C:\Users\Admin\AppData\Local\Temp\ipykernel_24820\2372565563.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("model.pth"))


<All keys matched successfully>